<a href="https://colab.research.google.com/github/Minyst/ML_DL_Portfolio_KR/blob/main/Detectron/Detectron2_Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [2]:
import cv2
import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances

# 사용자 파일 경로
input_video_path = '/content/PREMIER LEAGUE HIGHLIGHTS 10sec.mp4'
output_video_path = '/content/output_video.mp4'

In [ ]:
#한 frame의 shape 확인하기
cap = cv2.VideoCapture(input_video_path)
while cap.isOpened():
    ret, frame = cap.read()
    print(frame.shape)
    if ret:
        break
cap.release()

In [ ]:
# 모든 frame의 shape이랑 개수 확인하기
cap = cv2.VideoCapture(input_video_path)
cnt = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    print(frame.shape)
    cnt+=1
print(cnt)
cap.release()

In [5]:
#frame list에 담기
cap = cv2.VideoCapture(input_video_path)
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    frames.append(frame)
    if not ret:
        break

In [ ]:
# Config 및 모델 설정
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4  # threshold 설정
# 사전에 학습되어있는 COCO데이터 불러오는 것
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

In [ ]:
#예측한 하나의 frame shape확인하기
for frame in frames:
    outputs = predictor(frame)
    v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    result = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image = result.get_image()[:, :, ::-1]
    print(result_image.shape)
    break

width1 = result_image.shape[0]
height1 = result_image.shape[1]
print(width1, height1)

In [ ]:
#예측한 모든 frame shape확인하기
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

cnt = 0
result_images_lst = []

for frame in frames:
    if frame is None:
        break
    outputs = predictor(frame)
    v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    result = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image = result.get_image()[:, :, ::-1]
    print(result_image.shape)
    result_images_lst.append(result_image)
    cnt+=1

width2 = result_image.shape[0]
height2 = result_image.shape[1]
print(width2, height2)
print(cnt)

In [9]:
output_video_path = '/content/output_video.mp4'
cap = cv2.VideoCapture(input_video_path)
fps = cnt/10

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (height2, width2))

for result_image in result_images_lst:
    if result_image is None:
        break
    out.write(result_image)

cap.release()
out.release()